# Import Libraries

In [ ]:
import numpy as np
import os
import pylab
from pprint import pprint

# Load Signal

In [ ]:
# load signal
import os
from py_audiolib.io_tools import wavefile
dir0 = os.environ["DATA_FILES"]
file0 = dir0 + "/communications/AIS IQ.wav"
signal, info = wavefile.read_from_file(file0, float, False)
signal = signal[:,0] + 1j*signal[:,1]
pylab.specgram(signal, Fs=info["sample_rate"])
pylab.show()

# apply burst detection
from kcss.detectors.detector_energy import DetectorEnergy
det_energy = DetectorEnergy(100)
[bursts, sample_loc] = det_energy.analyze_signal(signal, True)
pprint(sample_loc)

# AIS
Automatic Identification System (AIS)

# Modulation
| Characteristic | Value |
| :-: | :-: |
| Frequency | 161.975 MHz and 162.025 MHz |
| Modulation | GMSK |
| Baud | 9600 symbols/s |

# Data Protocol

* The messages are HDLC encoded (with bit stuffing)
* Each message starts with a preamble of 24 bits of (0101...01)
* The bits are further NRZI encoded.

| Preamble | SB | Data | CRC | SB |
| :-: | :-:| :-: | :-: | :-: |
| 010101010101010101010101 | 01111110 | Bit stuffed data payload | 16 bit CRC-16-CCITT | 01111110 |





In [ ]:
from kcss.receivers.burst_receiver import BurstReceiver
from kcss.line_codes import line_code
from kcss.line_codes.hdlc import HDLC
from kcss.utils.binary import Binary
from kcss.utils.converter import bits_to_str

HDLC_FLAG = [0,1,1,1,1,1,1,0]
START_FLAG = np.array([0,1]*8 + HDLC_FLAG)


receiver = BurstReceiver()

fs = info["sample_rate"]
spec = {"baud": 9600.0, "symbol_map":[1, 0], "mod_index":0.5}
msgs = [None] * len(bursts)
burst_count = 0
for burst in bursts:
    print("="*50 + "\n")
    print("Burst = %d"%burst_count)
    burst_count += 1

    # demodulate GMSK
    c_demod_bits = receiver.fsk_receiver(burst, fs, spec)
    
    # NRZI decode
    c_bits = 1 - line_code.nrzi_decode(c_demod_bits)
    
    # ---------------  synchronize  ------------------
    # (search preamble + start flag, and end flag)
    start_ind = Binary.identify_location(
        c_bits, START_FLAG, threshold=14)
    
    #extract last occurence of HDLC flag
    end_ind = Binary.identify_location(
        c_bits, HDLC_FLAG, threshold=6, first=False)
    if start_ind and end_ind:
        synced_bits = c_bits[start_ind + len(START_FLAG):end_ind]
    else:
        print("start and/or end flag missing\n\n")
        continue

    # HDLC deframer
    hdlc_frame = HDLC.bit_destuff(synced_bits)
    
    # ----------------  print info  -------------------
    
    print("Message of length %d bits"%len(hdlc_frame))
    print(bits_to_str(hdlc_frame))
    msgs[burst_count-1] = hdlc_frame

# Messages

In [ ]:
from kcss.error_codes import cyclic_redundancy_check as CRCMOD
CCITT = CRCMOD.STANDARDS["CRC-16-CCITT"]

# load AIS message protocols
from kcss.protocol.data_packet import DataPacket as DP
AIS = DP.load_json("/home/keithchow/ais_message.json")

print(CCITT)
from kcss.info import numeric_info as NI
for msg in msgs:
    # -------------  skip Nones  --------------------------
    if msg is None:
        continue
        
    #
    print("="*70 + "\nMessage length = %d"%len(msg))

    # FIXME: Why don't CRC match?
    est_crc = CRCMOD.crc(msg[:-16].astype(np.int8), **CCITT)
    print("CRC parity = %s"%str(msg[-16:]))
    print("CRC estimate = %s"%str(NI.encode_bcd(est_crc, 16)))

    # print messages
    print("\n" + AIS[0].to_str(msg[:AIS[0].nbits]))